In [9]:
import yfinance as yf
import numpy as np
import pandas as pd


In [45]:
def monte_carlo_simulation(returns, num_simulations=1000, num_days=252):
    simulations = np.zeros((num_simulations, num_days))

    for i in range(num_simulations):
        daily_returns = returns.pct_change().dropna()
        avg_daily_return = daily_returns.mean()
        std_dev_daily_return = daily_returns.std()

        simulation = np.random.normal(avg_daily_return, std_dev_daily_return, num_days)
        simulations[i, :] = 1 + simulation

    return simulations


In [46]:
def filter_negative_correlation(stock_returns):
    correlation_matrix = stock_returns.corr()

    # Create a mask to filter out pairs with negative correlation coefficient higher than -0.9
    mask = (correlation_matrix < -0.9) & (correlation_matrix != 1)
    filtered_pairs = []

    for i in range(len(mask.columns)):
        for j in range(i):
            if mask.iloc[i, j]:
                filtered_pairs.append((mask.columns[i], mask.index[j]))

    return filtered_pairs


In [70]:
def get_filtered_stock_list():
    # Fetch all Indian stock symbols
    indian_stocks = yf.Tickers("*.BO")

    filtered_stocks = []

    for stock_ticker in indian_stocks.tickers:
        symbol = stock_ticker.info.get('symbol', '')
        market_cap = stock_ticker.info.get('marketCap', 0)
        if market_cap > 10000000000 and symbol:
            filtered_stocks.append(symbol)

    return filtered_stocks

In [48]:
def get_historical_data(symbol):
    stock_data = yf.download(symbol, start="2016-12-03", end="2023-12-03")  
    return stock_data['Adj Close']


In [59]:
def filter_negative_correlation(stock_returns):
    correlation_matrix = stock_returns.corr()

    # Create a mask to filter out pairs with negative correlation coefficient higher than -0.9
    mask = (correlation_matrix < -0.9) & (correlation_matrix != 1)
    filtered_pairs = []

In [65]:
 def calculate_correlation(filtered_stocks):
    all_correlation_pairs = []

    for i in range(len(filtered_stocks)):
        for j in range(i + 1, len(filtered_stocks)):
            symbol1 = filtered_stocks[i]
            symbol2 = filtered_stocks[j]

            data1 = get_historical_data(symbol1)
            data2 = get_historical_data(symbol2)

            # Assuming you want to calculate the correlation based on the daily returns
            returns1 = data1.pct_change().dropna()
            returns2 = data2.pct_change().dropna()

            correlation_coefficient = np.corrcoef(returns1, returns2)[0, 1]

            all_correlation_pairs.append((symbol1, symbol2, correlation_coefficient))
            

    return all_correlation_pairs


In [66]:
def filter_negative_correlation(stock_returns):
    correlation_matrix = stock_returns.corr()

    # Create a mask to filter out pairs with negative correlation coefficient higher than -0.9
    mask = (correlation_matrix < -0.9) & (correlation_matrix != 1)
    filtered_pairs = []


In [67]:
def filter_pairs(all_correlation_pairs, market_cap_threshold=10000000000, correlation_threshold=-0.9):
    filtered_pairs = [
        pair for pair in all_correlation_pairs
        if yf.Ticker(pair[0]).info.get('marketCap', 0) > market_cap_threshold
        and yf.Ticker(pair[1]).info.get('marketCap', 0) > market_cap_threshold
        and pair[2] < correlation_threshold
    ]

    return filtered_pairs
